In [1]:
import pandas as pd
import numpy as np

In [2]:
# 출력 옵션 변경

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1300)

In [3]:
dlvr_raw = pd.read_csv('../data/dlvr_call.csv');

c:\users\jeaung-lee\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
dlvr = dlvr_raw.copy()

In [5]:
dlvr.shape

(4296271, 32)

In [6]:
dlvr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4296271 entries, 0 to 4296270
Data columns (total 32 columns):
 #   Column                     Dtype 
---  ------                     ----- 
 0   SN                         int64 
 1   PROCESS_DT                 object
 2   DLVR_REQUST_ENTRPS_ID      object
 3   DLVR_REQUST_STTUS_VALUE    int64 
 4   DLVR_RCEPT_ENTRPS_ID       object
 5   DLVR_STORE_ID              object
 6   DLVR_STORE_INDUTY_NM       object
 7   DLVR_STORE_LEGALDONG_CODE  int64 
 8   DLVR_STORE_BRTC_NM         object
 9   DLVR_STORE_SIGNGU_NM       object
 10  DLVR_STORE_LEGALDONG_NM    object
 11  DLVR_STORE_LI_NM           object
 12  DLVR_STORE_ADSTRD_CODE     int64 
 13  DLVR_STORE_RDNMADR_CODE    object
 14  DLVR_DSTN_LEGALDONG_CODE   int64 
 15  DLVR_DSTN_BRTC_NM          object
 16  DLVR_DSTN_SIGNGU_NM        object
 17  DLVR_DSTN_LEGALDONG_NM     object
 18  DLVR_DSTN_LI_NM            object
 19  DLVR_DSTN_ADSTRD_CODE      object
 20  DLVR_DSTN_RDNMADR_CODE  

In [7]:
# 하나의 구 추출

dlvr = dlvr[dlvr['DLVR_STORE_SIGNGU_NM'] == '구로구']

In [8]:
dlvr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 357818 entries, 9 to 4296224
Data columns (total 32 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   SN                         357818 non-null  int64 
 1   PROCESS_DT                 357818 non-null  object
 2   DLVR_REQUST_ENTRPS_ID      357818 non-null  object
 3   DLVR_REQUST_STTUS_VALUE    357818 non-null  int64 
 4   DLVR_RCEPT_ENTRPS_ID       357818 non-null  object
 5   DLVR_STORE_ID              357818 non-null  object
 6   DLVR_STORE_INDUTY_NM       357818 non-null  object
 7   DLVR_STORE_LEGALDONG_CODE  357818 non-null  int64 
 8   DLVR_STORE_BRTC_NM         357818 non-null  object
 9   DLVR_STORE_SIGNGU_NM       357818 non-null  object
 10  DLVR_STORE_LEGALDONG_NM    357818 non-null  object
 11  DLVR_STORE_LI_NM           357818 non-null  object
 12  DLVR_STORE_ADSTRD_CODE     357818 non-null  int64 
 13  DLVR_STORE_RDNMADR_CODE    357818 non-null 

In [9]:
# 배달일시, 지역, 업종, 매출 컬럼 선택
# 배달일시 ['PROCESS_DT'], 행정동코드 ['DLVR_DSTN_ADSTRD_CODE'], 배달상점 업종이름 (카테고리) ['DLVR_STORE_INDUTY_NM'], 배달상품금액 ['GOODS_AMOUNT']

df_dlvr = dlvr[['PROCESS_DT','DLVR_STORE_INDUTY_NM','GOODS_AMOUNT']]

In [10]:
# 업종 컬럼 값 확인

df_dlvr['DLVR_STORE_INDUTY_NM'].unique()

array(['패스트푸드', '분식', '회', '치킨', '피자', '족발/보쌈', '카페/디저트', '한식', '중식',
       '야식', '찜탕', '돈까스/일식', '도시락', '배달전문업체', '아시안/양식'], dtype=object)

In [11]:
# 업종이'배달전문업체','심부름'인 행 삭제

df_dlvr.replace(['배달전문업체','심부름'], np.nan, inplace=True)
df_dlvr = df_dlvr.dropna(axis=0) 

c:\users\jeaung-lee\appdata\local\programs\python\python39\lib\site-packages\pandas\core\frame.py:4524: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [12]:
# 처리일시를 날짜 데이터 형태로 변경
df_dlvr['PROCESS_DT'] = pd.to_datetime(df_dlvr['PROCESS_DT'])

In [13]:
df_dlvr = df_dlvr.sort_values(by='PROCESS_DT', ascending=True)

In [14]:
df_daily = df_dlvr.groupby(by = ['PROCESS_DT', 'DLVR_STORE_INDUTY_NM']).sum()

In [15]:
df_daily

GOODS_AMOUNT
PROCESS_DT DLVR_STORE_INDUTY_NM              
2019-12-01 돈까스/일식                       50500
           분식                         1748000
           야식                          727000
           족발/보쌈                      2385100
           중식                          963600
...                                       ...
2020-05-31 카페/디저트                     2770201
           패스트푸드                      6913200
           피자                         3542390
           한식                         4531400
           회                           576000

[2485 rows x 1 columns]

In [16]:
# 정수형으로 변경(데이터 표기 통일)

df_daily.replace(np.nan, 0, inplace=True)
df_daily = df_daily.astype('int64')
df_daily.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2485 entries, (Timestamp('2019-12-01 00:00:00'), '돈까스/일식') to (Timestamp('2020-05-31 00:00:00'), '회')
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   GOODS_AMOUNT  2485 non-null   int64
dtypes: int64(1)
memory usage: 28.4+ KB


In [17]:
df_daily.to_csv('../data/df_daily.csv')

### 행정동별 생활인구 데이터

In [18]:
pop1 = pd.read_csv('../data/LOCAL_PEOPLE_DONG_201912.csv',header=0, index_col=False);
pop2 = pd.read_csv('../data/LOCAL_PEOPLE_DONG_202001.csv',header=0, index_col=False);
pop3 = pd.read_csv('../data/LOCAL_PEOPLE_DONG_202002.csv',header=0, index_col=False);
pop4 = pd.read_csv('../data/LOCAL_PEOPLE_DONG_202003.csv',header=0, index_col=False);
pop5 = pd.read_csv('../data/LOCAL_PEOPLE_DONG_202004.csv',header=0, index_col=False);
pop6 = pd.read_csv('../data/LOCAL_PEOPLE_DONG_202005.csv',header=0, index_col=False);

In [19]:
# 2019.12 ~ 2020.05 행정동별 생활인구 데이터 병합
pop = pd.concat([pop1,pop2,pop3,pop4,pop5,pop6])

In [20]:
df_pop = pop.copy()

In [21]:
df_pop

,기준일ID,시간대구분,행정동코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,남자35세부터39세생활인구수,남자40세부터44세생활인구수,남자45세부터49세생활인구수,남자50세부터54세생활인구수,남자55세부터59세생활인구수,남자60세부터64세생활인구수,남자65세부터69세생활인구수,남자70세이상생활인구수,여자0세부터9세생활인구수,여자10세부터14세생활인구수,여자15세부터19세생활인구수,여자20세부터24세생활인구수,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
0,20191201,0,11215710,45694.9988,452.3937,208.7971,1267.7678,5642.9235,5433.6801,2494.5993,1691.4831,1042.7096,1058.3378,895.4639,875.9212,601.9097,496.2165,850.6820,388.4005,180.3721,1586.7039,6973.6176,4259.7022,2036.1446,1285.4200,794.6241,833.7179,772.1417,773.2152,827.1455,565.5378,1405.3707
1,20191201,0,11680545,37375.4506,1298.5585,731.2901,889.5126,895.5983,1397.2099,1609.7025,1825.4062,1741.6219,1829.6326,1370.2235,1124.6595,878.5269,695.3232,1717.7044,1435.3327,787.0444,1130.2773,1215.9905,1628.3736,1503.9461,1752.1118,1591.0488,1555.4675,1033.6247,1094.5083,988.0638,944.5295,2710.1619
2,20191201,0,11680610,37266.9526,1083.2150,1559.2566,2339.8178,991.4434,990.1854,946.7321,1035.0291,1390.5989,2083.7386,1739.9398,1523.3575,1055.0147,899.4087,1534.7962,952.8383,1059.4625,1693.2841,959.5733,980.3396,888.9359,997.1139,1738.2101,2423.0246,1408.2536,1184.3355,1071.2366,858.9868,1878.8251
3,20191201,0,11680655,25231.2064,1191.0306,607.5946,639.2233,608.4116,697.5009,663.3373,919.4319,1074.9392,1228.4840,994.8977,765.8304,794.3326,659.9513,1110.7365,1067.8148,642.6462,862.7861,681.7974,767.0526,876.7431,1119.2400,1051.8991,1274.9860,852.3075,933.6367,949.1604,637.7794,1557.6550
4,20191201,0,11710570,40728.5564,1586.6273,818.9047,1204.9034,1172.2571,1195.7198,1163.9396,1472.8633,1347.5375,1603.9937,1482.1456,1575.2363,1455.9243,1080.4173,1650.7427,1459.9421,768.2610,1166.6483,1153.8815,1276.9370,1321.7052,1625.5710,1434.9812,1861.6645,1766.8905,1991.8166,1977.4260,1332.3513,2779.2676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315451,20200531,23,11500590,34850.2579,1462.5609,650.6488,751.8066,847.6573,1457.5636,1692.3361,1832.7683,1294.5065,1412.5338,989.9783,948.6449,827.1785,679.5684,1484.4242,1518.1599,587.3533,669.4936,1187.2936,1628.3094,1521.2461,1579.0513,1445.1335,1450.2348,1115.3927,1360.6946,1224.2473,1002.7457,2228.7270
315452,20200531,23,11710520,35815.8904,1522.6487,720.0644,783.0475,712.6163,1304.9236,1243.4551,1534.7862,1333.4719,1432.0687,1172.1130,1177.2657,1123.2123,835.0033,1543.8213,1327.6419,672.1250,931.3391,1293.4632,1699.0453,1569.7967,1621.0363,1411.4797,1646.5681,1158.6589,1462.9537,1414.7024,966.1372,2202.4438
315453,20200531,23,11410585,36944.2052,633.9095,199.2584,749.1463,3094.7758,2890.4101,1641.4097,1340.0429,829.3566,876.4381,751.4725,866.5132,649.7455,398.1159,1141.3060,309.0596,155.5619,1184.6523,6133.0721,3783.0008,1821.0791,1273.7274,873.9549,836.7297,839.4791,766.9024,733.7818,567.0416,1604.2628
315454,20200531,23,11410565,23996.6816,495.1475,317.0280,619.4652,953.5512,1081.7686,879.9429,1039.5685,842.3417,1135.0484,828.5651,862.8410,797.5366,500.6617,1076.5711,563.9444,356.7550,591.8719,1046.6147,1305.8847,938.6720,1057.4920,849.3335,915.2439,896.5478,981.3186,863.6758,516.9980,1682.2935


In [22]:
geo = pd.read_excel('../data/행정_법정동 중심좌표.xlsx',header=0, index_col=False);

In [24]:
geo.rename({'코드' : '행정동코드'}, axis=1, inplace=True)

,행정동코드,시도,시군구,읍면동,하위,위도,경도,코드종류
0,1111051500,서울특별시,종로구,청운효자동,NaN,37.584009,126.970626,H
1,1111053000,서울특별시,종로구,사직동,NaN,37.575408,126.965944,H
2,1111054000,서울특별시,종로구,삼청동,NaN,37.590758,126.980996,H
3,1111055000,서울특별시,종로구,부암동,NaN,37.594768,126.965574,H
4,1111056000,서울특별시,종로구,평창동,NaN,37.613029,126.974485,H
...,...,...,...,...,...,...,...,...
22214,5013032022,제주특별자치도,서귀포시,표선면,하천리,33.348068,126.832421,B
22215,5013032023,제주특별자치도,서귀포시,표선면,성읍리,33.393144,126.794822,B
22216,5013032024,제주특별자치도,서귀포시,표선면,가시리,33.354784,126.771072,B
22217,5013032025,제주특별자치도,서귀포시,표선면,세화리,33.321147,126.798867,B


In [25]:
geo = geo.drop(['하위'], axis = 1)

In [26]:
geo

,행정동코드,시도,시군구,읍면동,위도,경도,코드종류
0,1111051500,서울특별시,종로구,청운효자동,37.584009,126.970626,H
1,1111053000,서울특별시,종로구,사직동,37.575408,126.965944,H
2,1111054000,서울특별시,종로구,삼청동,37.590758,126.980996,H
3,1111055000,서울특별시,종로구,부암동,37.594768,126.965574,H
4,1111056000,서울특별시,종로구,평창동,37.613029,126.974485,H
...,...,...,...,...,...,...,...
22214,5013032022,제주특별자치도,서귀포시,표선면,33.348068,126.832421,B
22215,5013032023,제주특별자치도,서귀포시,표선면,33.393144,126.794822,B
22216,5013032024,제주특별자치도,서귀포시,표선면,33.354784,126.771072,B
22217,5013032025,제주특별자치도,서귀포시,표선면,33.321147,126.798867,B


In [ ]:
pd.merge(df_pop, geo, on='행정동코드', how='left')